In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

training_set = datagen.flow_from_directory(r"C:\Users\jigna\OneDrive\Desktop\ML Practical\train",
        target_size=(64, 64),
        batch_size=32,
        class_mode="binary"
      )

Found 557 images belonging to 2 classes.


In [3]:
datagen1 = ImageDataGenerator(rescale=1./255)

test_set = datagen1.flow_from_directory(r"C:\Users\jigna\OneDrive\Desktop\ML Practical\test",
        target_size=(64, 64),
        batch_size=32,
        class_mode="binary"
      )

Found 140 images belonging to 2 classes.


In [4]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

In [5]:
cnn = tf.keras.models.Sequential()

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', strides=2, input_shape=[64, 64, 3]))

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [9]:
cnn.add(tf.keras.layers.Flatten())

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [11]:
cnn.add(Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation ='linear'))

In [12]:
cnn.compile(optimizer = 'adam', loss = 'hinge', metrics = ['accuracy'])

In [13]:
r=cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 481ms/step - accuracy: 0.5246 - loss: 1.0289 - val_accuracy: 0.5000 - val_loss: 1.0052
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 253ms/step - accuracy: 0.5015 - loss: 1.0147 - val_accuracy: 0.5000 - val_loss: 1.0160
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 248ms/step - accuracy: 0.5007 - loss: 0.9930 - val_accuracy: 0.5071 - val_loss: 1.0092
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 268ms/step - accuracy: 0.5147 - loss: 0.9910 - val_accuracy: 0.5214 - val_loss: 0.9734
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.5400 - loss: 0.9458 - val_accuracy: 0.6071 - val_loss: 0.9653
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 273ms/step - accuracy: 0.5786 - loss: 0.8870 - val_accuracy: 0.5643 - val_loss: 0.9418
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 251ms/step - accuracy: 0.6295 - loss: 0.8449 - val_accuracy: 0.5357 - val_loss: 0.9475
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 277ms/step - accuracy: 0.5989 - loss: 0.8029 - val_accuracy: 0

In [16]:
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dog.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [17]:
if result[0]<0:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is dog


In [18]:
# SVM 
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt


In [20]:
def load_images_from_folder(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(label)
    return images, labels

cat_folder = r'C:\Users\jigna\OneDrive\Desktop\ML Practical\train\cats'  # Update this path
dog_folder = r'C:\Users\jigna\OneDrive\Desktop\ML Practical\train\dogs'  # Update this path

cat_images, cat_labels = load_images_from_folder(cat_folder, label=0)
dog_images, dog_labels = load_images_from_folder(dog_folder, label=1)

images = cat_images + dog_images
labels = cat_labels + dog_labels

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the images
images = images / 255.0


In [21]:
# Flatten the images
n_samples, height, width, n_channels = images.shape
images_flattened = images.reshape(n_samples, height * width * n_channels)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(images_flattened, labels, test_size=0.2, random_state=42)

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_scaled, y_train)

SVC(kernel='linear', random_state=42)

In [25]:
y_pred = svm.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.5625
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.61      0.61        64
           1       0.49      0.50      0.49        48

    accuracy                           0.56       112
   macro avg       0.55      0.55      0.55       112
weighted avg       0.56      0.56      0.56       112

